In [1]:
import flask
import io
import string
import time
import os
import pandas as pd
import numpy as np
from flask import Flask, jsonify, request
import joblib as joblib
import json

In [2]:
def read_preprocess(file):
    df = pd.read_excel(file)
    pd.set_option('display.max_rows', 15)
    df = df.iloc[14:70, :]
    df.drop(df.columns[[0, 4, 7, 8, 9, 10, 11, 12]], axis=1, inplace = True)
    df.drop(df.loc[:, 'Unnamed: 14':'Unnamed: 15'].columns, axis = 1, inplace = True)
    df.rename(columns={'Unnamed: 13': 'RMS(g)'}, inplace = True)
    df.dropna(subset = ['RMS(g)'], inplace = True)
    return df

def read_preprocess2(file):
    df = pd.read_excel(file)
    pd.set_option('display.max_rows', 15)
    df = df.iloc[72:85, :]
    df.drop(df.columns[[0, 4, 8, 9, 10, 11, 12]], axis=1, inplace=True)
    df.drop(df.loc[:, 'Unnamed: 13':'Unnamed: 15'].columns, axis = 1, inplace = True)
    df.rename(columns={'RMS': 'Slope of the line'}, inplace=True)
    return df

In [3]:
def predict(file, X_test):
    regressor = joblib.load(file) 
    y_predict = regressor.predict(X_test)
    return y_predict

def predict2(file, X_test):
    regressor2 = joblib.load(file)
    y_predict = regressor2.predict(X_test)
    return y_predict

In [4]:
app = Flask(__name__)

In [5]:
@app.route('/predict_rms', methods=['POST'])
def infer_image():
    # Catch the csv file from a POST request
    
    file_array = request.files.getlist("file[]")

    # Read and preprocess the csv file
    df = read_preprocess(file_array[0])

    # Return the prediction in JSON format
    X_test = df.iloc[:, 0:5]
    prediction = predict(file_array[1], X_test)
    lists = prediction.tolist()
    json_str = json.dumps(lists)
    return json_str

@app.route('/predict_slope', methods=['POST'])
def infer_image2():
    # Catch the csv file from a POST request
    
    file_array = request.files.getlist("file[]")

    # Read and preprocess the csv file
    df2 = read_preprocess2(file_array[0])

    # Return the prediction in JSON format
    X_test2 = df2.iloc[:, 0:5]
    prediction2 = predict2(file_array[1], X_test2)
    lists2 = prediction2.tolist()
    json_str2 = json.dumps(lists2)
    return json_str2
    

@app.route('/', methods=['GET'])
def index():
    return 'Machine Learning Inference'

In [6]:
if __name__ == '__main__':
    app.run(debug=False, host='0.0.0.0')

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on all addresses.
 * Running on http://192.168.43.242:5000/ (Press CTRL+C to quit)
192.168.43.242 - - [12/Jul/2021 21:29:39] "GET / HTTP/1.1" 200 -
[2021-07-12 21:29:58,872] ERROR in app: Exception on /predict_rms [POST]
Traceback (most recent call last):
  File "/home/rishabhgarg/.local/lib/python3.8/site-packages/flask/app.py", line 2070, in wsgi_app
    response = self.full_dispatch_request()
  File "/home/rishabhgarg/.local/lib/python3.8/site-packages/flask/app.py", line 1515, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/home/rishabhgarg/.local/lib/python3.8/site-packages/flask/app.py", line 1513, in full_dispatch_request
    rv = self.dispatch_request()
  File "/home/rishabhgarg/.local/lib/python3.8/site-packages/flask/app.py", line 1499, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**req.view_args)
  File "<ipython-input-5-d747f7b78034>", line 8, in infer_image
    df = read_preprocess(file_array[0])
In